In [48]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Sauvegarder le contenu HTML dans un fichier local
with open("page_content.html", "r", encoding="utf-8") as file:
    html_content = file.read()


# Analyser avec BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [64]:
# Trouver tous les <div> de class="visible"
from operator import concat

tabs = soup.select_one("div.tabs")

valuebets = concat(tabs.select("div.visible"), tabs.select("div.hidden"))


# Afficher le nombre d'elements dans valuebets[]
len(valuebets)

51

In [53]:
sports = []
countries = []
leagues = []


for valuebet in valuebets:
    # Recuperation de ce qui est sur le head 
    header = valuebet.select("a")

    sports.append(header[0].text.strip())
    countries.append(header[1].text.strip())
    leagues.append(header[2].text.strip().replace('\n                                                            ', ' '))
    
    
header_df = pd.DataFrame(data=[sports, countries, leagues]).transpose()
header_df.columns = ['sports', 'countries', 'leagues']
header_df.head()


,sports,countries,leagues
0,Football,Thailand,Thai League 1
1,Football,Brazil,Serie C
2,Football,Africa,CAF Champions League
3,Football,Brazil,Serie C
4,Football,Thailand,Thai League 1


In [73]:
pronos = []
date = []
time = []
team_1 = []
team_2 = []
outcome = []
odds = []
value = []
probability = []


match_infos = tabs.find_all('div', class_='flex min-h-[90px] w-full')

for match_info in match_infos:
    # Recuperation du match-info
    all_p = match_info.find_all('p')
    
    pronos.append(all_p[0].text.strip())
    date.append(all_p[1].text.strip())
    time.append(all_p[2].text.strip())
    team_1.append(all_p[3].text.strip())
    team_2.append(all_p[4].text.strip())
    outcome.append(all_p[5].text.strip())
    odds.append(all_p[6].text.strip())
    value.append(all_p[7].text.strip())
    probability.append(all_p[8].text.strip())

match_info_df = pd.DataFrame(data=[pronos, date, time, team_1, team_2, outcome, odds, value, probability]).transpose()
match_info_df.columns = ['pronos', 'date', 'time', 'team_1', 'team_2', 'outcome', 'odds', 'value', 'probability']
match_info_df.head()

,pronos,date,time,team_1,team_2,outcome,odds,value,probability
0,"Home/Away, FT including OT","26 Apr,",02:00,Montreal Canadiens,Washington Capitals,1,6.50,3.13,46.49%
1,"O/U 4.5, 1st Half","27 Apr,",14:00,Muang Thong Utd,Nakhon Ratchasima FC,Over,41.00,2.60,7.65%
2,"O/U 3.5, 1st Half","27 Apr,",01:30,CSA,Nautico,Over,34.00,2.45,7.91%
3,"O/U 3.5, 1st Half","Tomorr.,",21:00,Pyramids,Orlando Pirates,Over,34.00,2.42,7.80%
4,"O/U 3.5, 1st Half","27 Apr,",22:30,Itabaiana,Sao Bernardo,Over,34.00,2.42,7.78%


In [78]:
full_df = pd.concat([header_df, match_info_df], axis=1)
full_df.head()

,sports,countries,leagues,pronos,date,time,team_1,team_2,outcome,odds,value,probability
0,Football,Thailand,Thai League 1,"Home/Away, FT including OT","26 Apr,",02:00,Montreal Canadiens,Washington Capitals,1,6.50,3.13,46.49%
1,Football,Brazil,Serie C,"O/U 4.5, 1st Half","27 Apr,",14:00,Muang Thong Utd,Nakhon Ratchasima FC,Over,41.00,2.60,7.65%
2,Football,Africa,CAF Champions League,"O/U 3.5, 1st Half","27 Apr,",01:30,CSA,Nautico,Over,34.00,2.45,7.91%
3,Football,Brazil,Serie C,"O/U 3.5, 1st Half","Tomorr.,",21:00,Pyramids,Orlando Pirates,Over,34.00,2.42,7.80%
4,Football,Thailand,Thai League 1,"O/U 3.5, 1st Half","27 Apr,",22:30,Itabaiana,Sao Bernardo,Over,34.00,2.42,7.78%


In [82]:
full_df.to_csv('oddsportal_data_brut.csv', index=False)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sports       51 non-null     object
 1   countries    51 non-null     object
 2   leagues      51 non-null     object
 3   pronos       51 non-null     object
 4   date         51 non-null     object
 5   time         51 non-null     object
 6   team_1       51 non-null     object
 7   team_2       51 non-null     object
 8   outcome      51 non-null     object
 9   odds         51 non-null     object
 10  value        51 non-null     object
 11  probability  51 non-null     object
dtypes: object(12)
memory usage: 4.9+ KB


In [81]:
full_df['date'].unique()

array(['26 Apr,', '27 Apr,', 'Tomorr.,', '28 Apr,', '10 May,', '29 Apr,',
       '12 Jul,', '08 Jun,', '01 Jun,', '28 May,', '24 May,', '05 May,',
       '30 Aug,'], dtype=object)